**Note**: To automatically get the current hostname from the frontend, do not "Restart and Run All": make sure the following JS cell is executed and output *before* subsequent cells are run. Alternatively, set the variable `LOCATION` to the current hostname manually in a Python cell.

In [ ]:
%%javascript

IPython.notebook.kernel.execute('LOCATION = "' + window.location.href + '"');

In [ ]:
import numpy as np


def get_host():
    from urllib.parse import urlsplit
    parts = urlsplit(LOCATION)
    return parts.netloc.split(':')[0]


def lena():
    import requests
    import pickle
    from io import BytesIO
    r = requests.get('https://github.com/minrk/scipy-1/blob/master/scipy/misc/lena.dat?raw=true')
    lena = np.array(pickle.load(BytesIO(r.content)))
    return lena.astype(float)

In [ ]:
get_host()

In [ ]:
from higlass.widgets import HiGlassDisplay
from higlass.tilesets import Tileset
import hgtiles.npmatrix

data1 = lena()
ts1 = Tileset(
    name="Lena",
    datatype="matrix",
    tileset_info=lambda: hgtiles.npmatrix.tileset_info(data1),
    tiles=lambda tids: hgtiles.npmatrix.tiles_wrapper(data1, tids)
)

data2 = np.random.rand(1000, 1000)
ts2 = Tileset(
    name="random",
    datatype="matrix",
    tileset_info=lambda: hgtiles.npmatrix.tileset_info(data2),
    tiles=lambda tids: hgtiles.npmatrix.tiles_wrapper(data2, tids)
)

ts1.uuid

In [ ]:
import higlass.server as hgserver
import higlass.client as hgclient

PORT = 8888
server = hgserver.Server(tilesets=[ts1, ts2], host=get_host(), port=PORT)
server.start(log_file='/tmp/hgserver.log')
server.api_address

In [ ]:
server.api_address

In [ ]:
conf = hgclient.ViewConf([
    hgclient.View([
        hgclient.Track(track_type='top-axis', position='top'),
        hgclient.Track(track_type='left-axis', position='left'),
        hgclient.Track(track_type='left-axis', position='right'),
        hgclient.Track(track_type='top-axis', position='bottom'),
        hgclient.Track(
            track_type='heatmap',
            position='center',
            tileset=ts1, 
            api_url=server.api_address,
            height=450,
            options={
                'name': 'Lena' ,
                'valueScaleMin': 0, 
                'valueScaleMax': 256,
                "heatmapValueScaling": "linear",      
                 "colorRange": [
                    "rgba(0,0,0,1.0)",
                    "rgba(21,83,76,1.0)",
                    "rgba(162,121,74,1.0)",
                    "rgba(199,180,238,1.0)",
                    "rgba(256,256,256,1.0)"
                  ],
                }),
    ])
])
d = conf.to_dict()
d["trackSourceServers"] = [server.api_address]

HiGlassDisplay(viewconf=d)

You can also view this in full screen by loading the following JSON config into https://higlass.io

In [ ]:
import json
print(json.dumps(d, indent=2))